# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [5]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [6]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [7]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

display(prod_cust_pivot.head())
display(cust_prod_pivot.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.000000,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.013298,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.0,0.015873,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012987,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012987,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.015385,0.015385,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014925,0.0
412,0.0,0.0,0.0,0.0,0.013699,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.013699,0.013699,0.013699,0.0,0.0,0.0,0.000000,0.000000,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [8]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [12]:
# Crea un diccionario vacío
recommendations = {}

# Crea una lista con todos los clientes distintos
customers = list(customer_products['CustomerID'].unique())

#Para cada cliente:
for customer in customers:
    
    # Crea una lista con los 5 clientes más parecidos
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    
    # Filtra el dataframe agrupado por clientes y productos, por los clientes más parecidos
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    
    # Agrupa todos los productos sumando sus compras.
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    
    # Ordena los productos de mayor a menor comprado.
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    # Une el dataframe ordenado con el dataframe comparativo entre clientes y productos
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    
    # Renombra las columnas
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    
    # Filtra por los registros en los que el cliente no haya comprado el producto
    recs = merged[merged['Purchased']==0].head()
    
    # Recopila las recomendaciones en el diccionario vacío
    recommendations[customer] = list(recs['ProductName'])

# Crea un dataframe a partir del diccionario
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()

# Renombra las columnas
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [10]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [11]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [12]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [72]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [73]:
# Creo el dataframe que agrupa clientes y productos
customer_products = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": sum}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [74]:
# Creo el dataframe que compara clientes con productos
products_customer_pt = pd.pivot_table(
    customer_products, values = "Quantity", index = "ProductName", columns = "CustomerID").fillna(0)

products_customer_pt.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
# Normalizo por productos para poder detectar productos comprados por clientes similares, 
# quitando importancia a las cantidades de producto compradas. 

products_customer_pt = products_customer_pt.div(products_customer_pt.sum(axis = 1), axis = 0)
products_customer_pt.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.000000,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.013298,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [76]:
# Creo una similarity matrix para encontrar distancias entre los productos.

from scipy.spatial.distance import pdist, squareform

products = set(data["ProductName"])

products_dist = pd.DataFrame(1 / (1 + squareform(pdist(products_customer_pt, "euclidean"))), 
                                index = products, columns = products)

display(products_dist.head())

,Ketchup - Tomato,Bread - Hot Dog Buns,Pate - Cognac,Apricots - Halves,Brandy - Bar,"Bread - Roll, Canadian Dinner",Onions - Cippolini,"Spoon - Soup, Plastic",Creme De Banane - Marie,Beef - Rib Eye Aaa,...,Beef - Montreal Smoked Brisket,Sponge Cake Mix - Chocolate,Lambcasing,"Wine - Red, Cooking",Lettuce - Frisee,Pasta - Cheese / Spinach Bauletti,Chips Potato All Dressed - 43g,Wine - Fume Blanc Fetzer,Tea - Decaf Lipton,Tuna - Salad Premix
Ketchup - Tomato,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
Bread - Hot Dog Buns,0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Pate - Cognac,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Apricots - Halves,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Brandy - Bar,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477


### Step 2: Get the products purchased for a specific customer of your choice.

In [77]:
# Creo una lista con los productos comprados por el cliente 200.

products200 = [products_customer_pt.index[i] for i in range(len(products_customer_pt.index)) 
                if products_customer_pt[200][i] > 0]

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [78]:
# Para cada producto comprado por el cliente 200, creo un dataframe con los 5 productos 
# más parecidos. Meto esos productos en una lista. 

similar_products = []

for i in products200:
    similar_prods = pd.DataFrame(products_dist[i].sort_values(ascending = False)[1:6])
    for e in similar_prods.index:
        similar_products.append(e)

In [79]:
# Creo una nueva lista donde meto los productos de la lista anterior que no se encuentren también
# en la lista de productos comprados por el cliente 200.

sim_not_purchased = [prods for prods in similar_products if prods not in products200]

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [80]:
from collections import Counter

In [82]:
# Obtengo los 5 productos más repetidos en la lista de productos populares sin comprar.

most_popular = Counter(sim_not_purchased)
top5 = [product[0] for product in most_popular.most_common(5)]
        
top5

['Skirt - 29 Foot',
 'Wine - Red, Harrow Estates, Cab',
 'Mussels - Frozen',
 'Wine - Fume Blanc Fetzer',
 'Table Cloth 62x114 White']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [83]:
recommendations = {}

customers = list(set(data["CustomerID"]))

In [94]:
def obtain_rec_products(customer):
    
    products = [products_customer_pt.index[i] for i in range(len(products_customer_pt.index)) 
                if products_customer_pt[customer][i] > 0]

    similar_products = []
    for i in products:
        similar_prods = pd.DataFrame(products_dist[i].sort_values(ascending = False)[1:6])
        for e in similar_prods.index:
            similar_products.append(e)
            
    sim_not_purchased = [prods for prods in similar_products if prods not in products]
    
    
    most_popular = Counter(sim_not_purchased)

    top5 = [product[0] for product in most_popular.most_common(5)]
    
    recommendations[customer] = top5

In [95]:
for customer in customers:
    obtain_rec_products(customer)

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [96]:
recommendationsdf = pd.DataFrame(recommendations).T.reset_index()
recommendationsdf.columns = ["CustomerId", "Rec1", "Rec2", "Rec3", "Rec4", "Rec5"]
recommendationsdf.head()

,CustomerId,Rec1,Rec2,Rec3,Rec4,Rec5
0,83973,Mussels - Frozen,Table Cloth - 53x69 Colour,Clam Nectar,Rabbit - Whole,"Cheese - Boursin, Garlic / Herbs"
1,59399,Clam Nectar,Table Cloth - 53x69 Colour,"Wine - Red, Harrow Estates, Cab",Beef - Top Sirloin,"Doilies - 5, Paper"
2,92168,Clam Nectar,"Cheese - Boursin, Garlic / Herbs",Table Cloth - 53x69 Colour,Mussels - Frozen,Skirt - 29 Foot
3,49159,Table Cloth - 53x69 Colour,"Doilies - 5, Paper",Clam Nectar,"Wine - Red, Harrow Estates, Cab",Mustard Prepared
4,18441,Clam Nectar,Table Cloth - 53x69 Colour,Mustard Prepared,Wine - Fume Blanc Fetzer,"Wine - Red, Harrow Estates, Cab"


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [189]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

new_customerdf = pd.DataFrame(new_customer, index=["new_customer"]).T
new_customerdf.head()

,new_customer
Cookies - Assorted,3
Flavouring - Orange,3
Fenngreek Seed,1
Wine - White Cab Sauv.on,1
Bandage - Flexible Neon,3


### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [190]:
#data.head()

In [224]:
# Groupby

customer_products = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": sum}).reset_index()
customer_products.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [195]:
# Pivot Table

customer_productspt = customer_products.pivot_table(
    values = "Quantity", index = "CustomerID", columns = "ProductName").fillna(0).T

# Uno los datos del nuevo cliente
customer_productspt = customer_productspt.join(new_customerdf).fillna(0)

# Normalizo para restar importancia a la cantidad de compras de un producto
customer_productspt = customer_productspt.div(customer_productspt.sum(axis = 1), axis = 0)

customer_productspt.head()

,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new_customer
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [206]:
# Creo la similarity matrix, para poder encontrar los 5 clientes más parecidos al nuevo,
# y así recomendarle después los 5 productos populares que no tiene aun.

cust_dist = pd.DataFrame(1/(1 + squareform(pdist(customer_productspt.T, 'euclidean'))),
                         index=customer_productspt.columns.values, columns=customer_productspt.columns.values)

In [223]:
# Obtengo los 5 clientes más parecidos al nuevo

similar_custs = cust_dist.sort_values("new_customer", ascending = False)[1:6].index
similar_custs

Index([1072, 3535, 412, 2503, 3472], dtype='object')

In [228]:
# Filtro el dataframe que agrupa clientes y productos por los clientes parecidos al nuevo

similar_customers = customer_products[customer_products["CustomerID"].isin(similar_custs)]
similar_customers = similar_customers.groupby(
    "ProductName").agg({"Quantity": sum}).sort_values("Quantity", ascending = False)

similar_customers.head()

,Quantity
ProductName,
Foam Dinner Plate,4
Spinach - Baby,4
Milk - 2%,3
Sauce - Demi Glace,3
"Cheese - Brie, Triple Creme",3


In [234]:
recommended_products = []

for product in similar_customers.index:
    if product not in new_customer:
        if len(recommended_products) < 5:
            recommended_products.append(product)
            
print(recommended_products)

['Foam Dinner Plate', 'Milk - 2%', 'Sauce - Demi Glace', 'Cheese - Brie, Triple Creme', 'Juice - Lime']


### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [259]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

products = [prod for prod in new_customer]
quantities = [new_customer[prod] for prod in new_customer]
new_customer = {"CustomerID": 0, "ProductName": products, "Quantity": quantities}

new_customerdf = pd.DataFrame(new_customer)
new_customerdf

,CustomerID,ProductName,Quantity
0,0,Cookies - Assorted,3
1,0,Flavouring - Orange,3
2,0,Fenngreek Seed,1
3,0,Wine - White Cab Sauv.on,1
4,0,Bandage - Flexible Neon,3
5,0,Oil - Shortening - All - Purpose,2
6,0,Beef - Montreal Smoked Brisket,4
7,0,French Pastry - Mini Chocolate,4
8,0,Snapple Lemon Tea,5
9,0,"Pepper - White, Ground",2


In [265]:
# Groupby
prod_cust = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": sum}).reset_index()

# Meto el nuevo cliente
prod_cust = pd.merge(prod_cust, new_customerdf, how = "outer")

prod_cust.sort_values("CustomerID").head(15)

,CustomerID,ProductName,Quantity
63639,0,"Sole - Dover, Whole, Fresh",4
63638,0,Spinach - Baby,5
63629,0,Flavouring - Orange,3
63630,0,Fenngreek Seed,1
63631,0,Wine - White Cab Sauv.on,1
63632,0,Bandage - Flexible Neon,3
63628,0,Cookies - Assorted,3
63634,0,Beef - Montreal Smoked Brisket,4
63635,0,French Pastry - Mini Chocolate,4
63636,0,Snapple Lemon Tea,5


In [271]:
# Pivot table
prod_cust_pt = prod_cust.pivot_table(values = "Quantity", index = "ProductName", columns = "CustomerID").fillna(0)

# Normalizo
prod_cust_pt = prod_cust_pt.div(prod_cust_pt.sum(axis = 1), axis = 0)
prod_cust_pt.head()

CustomerID,0,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,...,0.000000,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.011221,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,...,0.013298,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0
Appetizer - Sausage Rolls,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0
Apricots - Dried,0.0,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,...,0.000000,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [275]:
# Sim matrix

products_dist = pd.DataFrame(1 / (1 + squareform(pdist(prod_cust_pt, "euclidean"))), 
                             index = prod_cust_pt.index, columns = prod_cust_pt.index)

products_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477


In [277]:
# Ya tengo la lista de productos comprados por el nuevo cliente y la similarity matrix de productos.
# Tengo que encontrar los 5 productos más similares a cada producto del cliente, quitar los que ya ha comprado,
# y encontrar los 5 más populares.

products_new_customer = products

similar_products = []

for prod in products_new_customer:
    
    similar_prods = pd.DataFrame(products_dist[prod].sort_values(ascending = False)[1:6])
    for e in similar_prods.index:
         similar_products.append(e)
            
similar_notpurch = []

for prod in similar_products:
    if prod not in products_new_customer:
        similar_notpurch.append(prod)
        
print(len(products_new_customer))
print(len(similar_products))
print(len(similar_notpurch))

12
60
49


In [278]:
most_popular = Counter(similar_notpurch)
top5 = [product[0] for product in most_popular.most_common(5)]
        
top5

['Beef - Chuck, Boneless',
 'Towels - Paper / Kraft',
 'Rice - Jasmine Sented',
 'Hersey Shakes',
 'Rosemary - Primerba, Paste']